<a href="https://colab.research.google.com/github/JaimeLopezGarrido/labo2025v/blob/main/src/ensembles/594_TareaHogar_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarea para el Hogar 05

Esta Tarea para el Hogar 05 se entrega el final de la cuarta clase
<br> se espera de usted que intente avanzar con los desafios propuestos y que los traiga terminados para la Clase 05 que será el miercoles 03 de septiembre

##  1. Overfitting the Public Leaderboard

Leer  https://medium.com/hmif-itb/overfitting-the-leaderboard-da25172ac62e
( 8 minutos )

## 2. Hiperparámetros del LightGBM

Los objetivos de esta tarea son:


*   Aumentar la rentabilidad de la campaña de marketing de retención proactiva de clientes.
*   Generar un mejor modelo optimizando sus hiperparámetros
*   Conceptual : investigar los mas relevantes hiperparámetros de LightGBM
*   Familiarizarse con la Bayesian Optimization, sus largos tiempos de corrida y opciones para reducirlos
*   Familiarizarse con el uso de máquinas virtuales de Google Colab
*   Ver un pipeline completo de optimización de hiperparámetros y puesta en producción

LightGBM cuenta con mas de 60 hiperparámetros, siendo posible utilizar 40 al mismo tiempo, aunque no razonable.
<br> La documentación oficial de los hiperparámetros de LightGBM es  https://lightgbm.readthedocs.io/en/latest/Parameters.html#core-parameters


Se lo alerta sobre que una Optimizacion Bayesiana lleva varias horas de corrida, y usted deberá correr VARIAS optimizaciones para descubrir cuales parámetros conviene optimizar.
<br> A pesar que la próxima clase es recien en viernes 01 de agosto, inicie la tarea con tiempo, aprenda a planificar estratégicamente sus corridas como un@ científ@  de datos.

Es necesario investigar cuales son los hiperparámetros de LightGBM que vale la pena optimizar en una Bayesian Optimization, ya que los realmente utiles son apenas un reducido subconjunto.
<br>Usted deberá investigar cuales son los hiperparámetros mas relevantes de LightGBM, su primer alternativa es preguntándole a su amigo con capacidades especiales ChatGPT o sus endogámicos familiares Claude, DeepSeek, Gemini, Grok, etc
<br> La segunda alternativa es la propia documentación de LightGBM  https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html


Adicionalmente podra buscar información como la que proveen esta diminuta muestra aleatoria de artículos ligeros:
* https://machinelearningmastery.com/light-gradient-boosted-machine-lightgbm-ensemble/
*  https://medium.com/@sarahzouinina/a-deep-dive-into-lightgbm-how-to-choose-and-tune-parameters-7c584945842e
*  https://www.kaggle.com/code/somang1418/tuning-hyperparameters-under-10-minutes-lgbm
*  https://towardsdatascience.com/beginners-guide-to-the-must-know-lightgbm-hyperparameters-a0005a812702/


<br>  La muestra anterior se brinda a modo de ejemplo, usted deberá buscar muuuuchas  fuentes adicionales de información
<br> Tenga presente que LightGBM es el estado del arte en modelado predictivo para datasets estructurado, que son el 90% del trabajo del 95% de los Data Scientists en Argentina.

El desafío de esta tarea es:
* Qué hiperparparámetros conviene optimizar?  Las recomendaciones de los artículos ligeros es siempre sensata?  Sus autores realmente hicieron experimentos o son siemplemente escritores de entretenimiento carente de base científica?
* Elegidos los hiperparámetros, cual es el  <desde, hasta> que se debe utilizar en la Bayesian Optimization ?
* Realmente vale la pena optimizar 10 o 16 hiperparámetros al mismo tiempo ?  No resulta contraproducente una búsqueda en un espacio de tal alta dimensionalidad ?



---
# Resumen Hiperparametros

LightGBM utiliza el algoritmo de crecimiento de árbol "leaf-wise" (por hoja), mientras que muchas otras herramientas populares utilizan el crecimiento "depth-wise" (por profundidad).

Parámetros Importantes para el Árbol "Leaf-wise":

- *num_leaves*  = Número máximo de hojas que cada árbol puede tener; controla complejidad.

      Este es el parámetro principal para controlar la complejidad del modelo de árbol. Una profundidad no restringida puede inducir el sobreajuste.$num\_leaves = 2^{(max\_depth)}$ para obtener el mismo número de hojas que un árbol "depth-wise".

      - Tiene que ser menor que 2^{(max\_depth)} para evitar overfitting.
      - Disminuirlo acelera el entrenamiento y reduce overfitting.

- *min_data_in_leaf* = Número mínimo de observaciones que cada hoja debe contener.
      
      Muy importante para prevenir overfitting, ya que previene hojas con muy pocos datos. Cientos o miles es suficiente para un conjunto de datos grande.
      - Si sobreajusta hay que subirlo
      - Si subajusta hay que bajarlo

- *max_depth* = Profundidad máxima de cada árbol; ≤0 significa sin límite. Disminuirlo reduce tiempo y sobreajuste.

      -1 → sin límite (LightGBM lo decide por num_leaves)

      3–8 → árboles pequeños y conservadores

      8–15 → árboles medianos

      > 15 → árboles muy complejos, riesgo de sobreajuste



---
# Arboles, menos profundos, mas rapidos, pero menos precisos.


- Disminuir *max_depth*: Reducir la profundidad máxima del árbol para limitar el crecimiento y el tiempo de entrenamiento.
- Disminuir *num_leaves*: Reducir el número máximo de nodos hoja por árbol, controlando la complejidad y disminuyendo el tiempo de entrenamiento.
- Aumentar *min_gain_to_split*: Incrementar la ganancia mínima requerida para una división, evitando divisiones de mejora muy pequeña y acortando el entrenamiento.
- Aumentar *min_data_in_leaf* y *min_sum_hessian_in_leaf*: Aumentar estos valores para prevenir la creación de nodos de árbol muy específicos (sobreajuste) y reducir el tiempo de entrenamiento.

---

# Entrenar menos árboles
*num_iterations*

- Número de árboles o rondas de boosting.

- Valores típicos: 500–2000.

- Menos iteraciones → menos tiempo de entrenamiento.

- Si se reduce num_iterations, conviene subir learning_rate (por ejemplo, de 0.03 a 0.1).

*learning_rate*

- Cuánto se ajusta el modelo en cada iteración.

- Valores típicos: 0.01–0.1.

- No afecta el tiempo de entrenamiento, pero sí la precisión.

*early_stopping_round*

- Detiene el entrenamiento si no mejora tras X iteraciones.

- Ejemplo: early_stopping_round = 50.

- Requiere usar un set de validación.

- Permite ahorrar tiempo sin perder rendimiento.

---

# Menos divisiones por nodo
*feature_pre_filter*

Si está en TRUE (por defecto), LightGBM filtra variables con poca información antes de entrenar. Mantenerlo en TRUE para acelerar.

*max_bin*

- Cantidad máxima de bins por variable continua.

- Menor valor = entrenamiento más rápido pero menos preciso.

- Valores típicos: 31–255.

- Reducirlo acorta el tiempo de entrenamiento.

*min_data_in_bin*

- Mínimo de datos por bin.

- Aumentarlo reduce el número de bins y acelera el proceso.

- Valor típico: 3–10.

*feature_fraction*

- Porcentaje de variables usadas en cada árbol.

- Ejemplo: feature_fraction = 0.5 usa el 50% de las variables al azar.

- Reducirlo mejora velocidad y ayuda a evitar sobreajuste.

- Valores típicos: 0.5–0.9.

*max_cat_threshold*

- Número máximo de combinaciones evaluadas para variables categóricas.

- Reducirlo disminuye tiempo de cómputo.

- Valores comunes: 16–64.

---


# Usar menos datos por iteración
*bagging_fraction* y *bagging_freq*

Permiten usar una muestra aleatoria del dataset en cada iteración.

Ejemplo:

- bagging_fraction = 0.75 → usa el 75% de los datos.

- bagging_freq = 5 → cambia la muestra cada 5 iteraciones.

Reducir bagging_fraction acelera el entrenamiento y puede mejorar generalización.

---

# Para una Mejor Precisión

Para maximizar la precisión del modelo (generalmente a costa de un mayor tiempo de entrenamiento):

- Usar un max_bin grande (puede ser más lento).

- Usar un learning_rate pequeño con un num_iterations grande.

- Usar un num_leaves grande (puede causar sobreajuste).

- Usar datos de entrenamiento más grandes.

- Probar el booster dart.

**Cómo Lidiar con el Sobreajuste**

Para ayudar al modelo a generalizar mejor y evitar el sobreajuste:

- Usar un max_bin pequeño.

- Usar un num_leaves pequeño.

- Usar min_data_in_leaf y min_sum_hessian_in_leaf.

- Usar bagging (agregación de bootstrap) configurando bagging_fraction y bagging_freq.

- Usar submuestreo de características (feature sub-sampling) configurando feature_fraction.

- Usar datos de entrenamiento más grandes.

- Probar lambda_l1, lambda_l2 y min_gain_to_split para regularización.

- Probar max_depth para evitar el crecimiento de árboles profundos.

- Probar el algoritmo extra_trees.

- Probar aumentar path_smooth.

#### 2.1  Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Type -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [1]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Mounted at /content/.drive


Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab



In [2]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets



archivo_origen="https://storage.googleapis.com/open-courses/austral2025-af91/dataset_pequeno.csv"
archivo_destino="/content/datasets/dataset_pequeno.csv"
archivo_destino_bucket="/content/buckets/b1/datasets/dataset_pequeno.csv"

if ! test -f $archivo_destino_bucket; then
  wget  $archivo_origen  -O $archivo_destino_bucket
fi


if ! test -f $archivo_destino; then
  cp  $archivo_destino_bucket  $archivo_destino
fi


### 2.2 Optimizacion Hiperparámetros

Esta parte se debe correr con el runtime en lenguaje R Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

### 2.2.1 Inicio

limpio el ambiente de R

In [1]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Sat Nov 01 02:54:55 PM 2025"

In [2]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,660381,35.3,1454462,77.7,1454462,77.7
Vcells,1226627,9.4,8388608,64.0,1975128,15.1


### 2.2.2 Carga de Librerias

In [3]:
# cargo las librerias que necesito
require("data.table")
require("parallel")

if( !require("primes") ) install.packages("primes")
require("primes")

if( !require("utils") ) install.packages("utils")
require("utils")

if( !require("rlist") ) install.packages("rlist")
require("rlist")

if( !require("yaml")) install.packages("yaml")
require("yaml")

if( !require("lightgbm") ) install.packages("lightgbm")
require("lightgbm")

if( !require("DiceKriging") ) install.packages("DiceKriging")
require("DiceKriging")

if( !require("mlrMBO") ) install.packages("mlrMBO")
require("mlrMBO")

Loading required package: data.table

Loading required package: parallel

Loading required package: primes

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘primes’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: primes

Loading required package: rlist

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘rlist’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘XML’


Loading required package: rlist

Loading required package: yaml

Loading required package: lightgbm

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘lightgbm’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading requ

### 2.2.3 Definicion de Parametros

aqui debe cargar SU semilla primigenia
<br>recuerde cambiar el numero de experimento en cada corrida nueva

In [4]:
PARAM <- list()
PARAM$experimento <- 6001
PARAM$semilla_primigenia <- 100007


In [5]:
PARAM$kaggle$competencia <- "labo-i-2025-virtual-analista-sr"
PARAM$kaggle$cortes <- seq(10000, 12000, by= 500)

In [6]:
# un undersampling de 0.1  toma solo el 10% de los CONTINUA
# undersampling de 1.0  implica tomar TODOS los datos

PARAM$trainingstrategy$undersampling <- 0.5

In [7]:
# Parametros LightGBM

PARAM$hyperparametertuning$xval_folds <- 5

# parametros fijos del LightGBM que se pisaran con la parte variable de la BO
PARAM$lgbm$param_fijos <-  list(
  boosting= "gbdt", # puede ir  dart  , ni pruebe random_forest
  objective= "binary",
  metric= "auc",
  first_metric_only= FALSE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  force_row_wise= TRUE, # para reducir warnings
  verbosity= -100,

  seed= PARAM$semilla_primigenia,

  max_depth= -1L, # -1 significa no limitar,  por ahora lo dejo fijo
  min_gain_to_split= 0, # min_gain_to_split >= 0
  min_sum_hessian_in_leaf= 0.001, #  min_sum_hessian_in_leaf >= 0.0
  lambda_l1= 0.0, # lambda_l1 >= 0.0
  lambda_l2= 0.0, # lambda_l2 >= 0.0
  max_bin= 31L, # lo debo dejar fijo, no participa de la BO

  bagging_fraction= 1.0, # 0.0 < bagging_fraction <= 1.0
  pos_bagging_fraction= 1.0, # 0.0 < pos_bagging_fraction <= 1.0
  neg_bagging_fraction= 1.0, # 0.0 < neg_bagging_fraction <= 1.0
  is_unbalance= FALSE, #
  scale_pos_weight= 1.0, # scale_pos_weight > 0.0

  drop_rate= 0.1, # 0.0 < neg_bagging_fraction <= 1.0
  max_drop= 50, # <=0 means no limit
  skip_drop= 0.5, # 0.0 <= skip_drop <= 1.0

  extra_trees= FALSE,

  num_iterations= 1200,
  learning_rate= 0.02,
  feature_fraction= 0.5,
  num_leaves= 750,
  min_data_in_leaf= 5000
)


Aqui se definen los hiperparámetros de LightGBM que participan de la Bayesian Optimization
<br> si es un numero entero debe ir  makeIntegerParam
<br> si es un numero real (con decimales) debe ir  makeNumericParam
<br> es muy importante leer cuales son un lower y upper  permitidos y ademas razonables

In [37]:
# Aqui se cargan los bordes de los hiperparametros de la BO
#PARAM$hypeparametertuning$hs <- makeParamSet(
#  makeNumericParam("learning_rate", lower = 0.01, upper = 0.05),
#  makeIntegerParam("num_leaves", lower = 16L, upper = 128L),
#  makeNumericParam("feature_fraction", lower = 0.5, upper = 0.9),
#  makeNumericParam("bagging_fraction", lower = 0.7, upper = 1.0),
#  makeIntegerParam("min_data_in_leaf", lower = 20L, upper = 200L),
#  makeNumericParam("lambda_l1", lower = 0, upper = 10),
#  makeNumericParam("lambda_l2", lower = 0, upper = 10),
#  makeNumericParam("min_gain_to_split", lower = 0, upper = 0.5),
#  makeIntegerParam("max_depth", lower = -1L, upper = 10L)
#)

PARAM$hypeparametertuning$hs <- makeParamSet(
  makeNumericParam("learning_rate", lower = 0.01, upper = 0.03),
  makeIntegerParam("num_leaves", lower = 32L, upper = 96L),
  makeNumericParam("feature_fraction", lower = 0.6, upper = 0.9),
  makeNumericParam("bagging_fraction", lower = 0.7, upper = 1.0),
  makeIntegerParam("bagging_freq", lower = 0L, upper = 10L),
  makeIntegerParam("min_data_in_leaf", lower = 50L, upper = 250L),
  makeNumericParam("lambda_l1", lower = 0, upper = 10),
  makeNumericParam("lambda_l2", lower = 0, upper = 10),
  makeNumericParam("min_gain_to_split", lower = 0, upper = 0.3),
  makeIntegerParam("max_depth", lower = -1L, upper = 10L),
  makeNumericParam("min_sum_hessian_in_leaf", lower = 0.0001, upper = 10),
  makeNumericParam("scale_pos_weight", lower = 10, upper = 200)
)

A mayor cantidad de hiperparámetros, se debe aumentar las iteraciones de la Bayesian Optimization
<br> 30 es un valor muy tacaño, pero corre rápido
<br> deberia partir de 50, alcanzando los 100 si se dispone de tiempo

In [38]:
PARAM$hyperparametertuning$iteraciones <- 35 # iteraciones bayesianas

### 2.2.4  Preprocesamiento

In [39]:
# carpeta de trabajo

setwd("/content/buckets/b1/exp")
experimento_folder <- paste0("HT", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

In [40]:
# lectura del dataset

dataset <- fread("/content/datasets/dataset_pequeno.csv")

In [41]:
dataset_train <- dataset[foto_mes %in% c(202107)]

In [42]:
# paso la clase a binaria que tome valores {0,1}  enteros
#  BAJA+1 y BAJA+2  son  1,   CONTINUA es 0

dataset_train[,
  clase01 := ifelse(clase_ternaria %in% c("BAJA+2","BAJA+1"), 1L, 0L)
]

In [43]:
# defino los datos que forma parte del training
# aqui se hace el undersampling de los CONTINUA
# notar que para esto utilizo la SEGUNDA semilla

set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset_train[, azar := runif(nrow(dataset_train))]
dataset_train[, training := 0L]

dataset_train[
  foto_mes %in% c(202107) &
    (azar <= PARAM$trainingstrategy$undersampling | clase_ternaria %in% c("BAJA+1", "BAJA+2")),
  training := 1L
]

In [44]:
# los campos que se van a utilizar

campos_buenos <- setdiff(
  colnames(dataset_train),
  c("clase_ternaria", "clase01", "azar", "training")
)

In [45]:
# dejo los datos en el formato que necesita LightGBM

dtrain <- lgb.Dataset(
  data= data.matrix(dataset_train[training == 1L, campos_buenos, with= FALSE]),
  label= dataset_train[training == 1L, clase01],
  free_raw_data= FALSE
)

nrow(dtrain)
ncol(dtrain)

[1] 83029

[1] 154

2.2.5 Configuracion Bayesian Optimization

In [46]:
# En el argumento x llegan los parmaetros de la bayesiana
#  devuelve la AUC en cross validation del modelo entrenado

EstimarGanancia_AUC_lightgbm <- function(x) {

  # x pisa (o agrega) a param_fijos
  param_completo <- modifyList(PARAM$lgbm$param_fijos, x)

  # entreno LightGBM
  modelocv <- lgb.cv(
    data= dtrain,
    nfold= PARAM$hyperparametertuning$xval_folds,
    stratified= TRUE,
    param= as.list(param_completo[names(param_completo) != "boost_from_average"]) # Ensure param is a named list/vector
  )

  # obtengo la ganancia
  AUC <- modelocv$best_score

  # hago espacio en la memoria
  rm(modelocv)
  gc(full= TRUE, verbose= FALSE)

  message(format(Sys.time(), "%a %b %d %X %Y"), " AUC ", AUC)

  return(AUC)
}

In [47]:
# Aqui comienza la configuracion de la Bayesian Optimization

# en este archivo quedan la evolucion binaria de la BO
kbayesiana <- "bayesiana.RDATA"

funcion_optimizar <- EstimarGanancia_AUC_lightgbm # la funcion que voy a maximizar

configureMlr(show.learner.output= FALSE)

# configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
# por favor, no desesperarse por lo complejo

obj.fun <- makeSingleObjectiveFunction(
  fn= funcion_optimizar, # la funcion que voy a maximizar
  minimize= FALSE, # estoy Maximizando la ganancia
  noisy= TRUE,
  par.set= PARAM$hypeparametertuning$hs, # definido al comienzo del programa
  has.simple.signature= FALSE # paso los parametros en una lista
)

# cada 600 segundos guardo el resultado intermedio
ctrl <- makeMBOControl(
  save.on.disk.at.time= 600, # se graba cada 600 segundos
  save.file.path= kbayesiana
) # se graba cada 600 segundos

# indico la cantidad de iteraciones que va a tener la Bayesian Optimization
ctrl <- setMBOControlTermination(
  ctrl,
  iters= PARAM$hyperparametertuning$iteraciones
) # cantidad de iteraciones

# defino el método estandar para la creacion de los puntos iniciales,
# los "No Inteligentes"
ctrl <- setMBOControlInfill(ctrl, crit= makeMBOInfillCritEI())

# establezco la funcion que busca el maximo
surr.km <- makeLearner(
  "regr.km",
  predict.type= "se",
  covtype= "matern3_2",
  control= list(trace= TRUE)
)


2.2.6 Corrida Bayesian Optimization

In [48]:
# inicio la optimizacion bayesiana, retomando si ya existe
# es la celda mas lenta de todo el notebook

if (!file.exists(kbayesiana)) {
  bayesiana_salida <- mbo(obj.fun, learner= surr.km, control= ctrl)
} else {
  bayesiana_salida <- mboContinue(kbayesiana) # retomo en caso que ya exista
}

Computing y column(s) for design. Not provided.

Sat Nov 01 03:20:36 PM 2025 AUC 0.924989975139509

Sat Nov 01 03:24:13 PM 2025 AUC 0.925875229333851

Sat Nov 01 03:26:14 PM 2025 AUC 0.925724504910479

Sat Nov 01 03:30:10 PM 2025 AUC 0.926992230818329

Sat Nov 01 03:32:05 PM 2025 AUC 0.919204477518979

Sat Nov 01 03:34:22 PM 2025 AUC 0.925409655609331

Sat Nov 01 03:37:22 PM 2025 AUC 0.925791252896511

Sat Nov 01 03:41:27 PM 2025 AUC 0.923927993140774

Sat Nov 01 03:44:14 PM 2025 AUC 0.927982869044679

Sat Nov 01 03:47:34 PM 2025 AUC 0.924411383120829

Sat Nov 01 03:52:46 PM 2025 AUC 0.928768668053785

Sat Nov 01 03:56:08 PM 2025 AUC 0.927440662340898

Sat Nov 01 03:59:00 PM 2025 AUC 0.928150015425845

Sat Nov 01 04:02:17 PM 2025 AUC 0.927771790959397

Sat Nov 01 04:04:51 PM 2025 AUC 0.9254416719633

Sat Nov 01 04:06:50 PM 2025 AUC 0.927025869579548

Sat Nov 01 04:11:26 PM 2025 AUC 0.927609620210988

Sat Nov 01 04:13:35 PM 2025 AUC 0.924462095197052

Sat Nov 01 04:16:07 PM 2025 AUC 0.9

In [49]:

tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)
colnames( tb_bayesiana)

[1] "learning_rate"           "num_leaves"             
 [3] "feature_fraction"        "bagging_fraction"       
 [5] "bagging_freq"            "min_data_in_leaf"       
 [7] "lambda_l1"               "lambda_l2"              
 [9] "min_gain_to_split"       "max_depth"              
[11] "min_sum_hessian_in_leaf" "scale_pos_weight"       
[13] "y"                       "dob"                    
[15] "eol"                     "error.message"          
[17] "exec.time"               "ei"                     
[19] "error.model"             "train.time"             
[21] "prop.type"               "propose.time"           
[23] "se"                      "mean"

In [50]:
# almaceno los resultados de la Bayesian Optimization
# y capturo los mejores hiperparametros encontrados

tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)

tb_bayesiana[, iter := .I]

# ordeno en forma descendente por AUC = y
setorder(tb_bayesiana, -y)

# grabo para eventualmente poder utilizarlos en OTRA corrida
fwrite( tb_bayesiana,
  file= "BO_log.txt",
  sep= "\t"
)

# los mejores hiperparámetros son los que quedaron en el registro 1 de la tabla
PARAM$out$lgbm$mejores_hiperparametros <- tb_bayesiana[
  1, # el primero es el de mejor AUC
  setdiff(colnames(tb_bayesiana),
    c("y","dob","eol","error.message","exec.time","ei","error.model",
      "train.time","prop.type","propose.time","se","mean","iter")),
  with= FALSE
]


PARAM$out$lgbm$y <- tb_bayesiana[1, y]


In [51]:
write_yaml( PARAM, file="PARAM.yml")

In [52]:
print(PARAM$out$lgbm$mejores_hiperparametros)
print(PARAM$out$lgbm$y)

   learning_rate num_leaves feature_fraction bagging_fraction bagging_freq
           <num>      <int>            <num>            <num>        <int>
1:    0.01005514         51        0.7063844        0.9783509            5
   min_data_in_leaf lambda_l1 lambda_l2 min_gain_to_split max_depth
              <int>     <num>     <num>             <num>     <int>
1:              157  9.632635  7.882127        0.07555869         0
   min_sum_hessian_in_leaf scale_pos_weight
                     <num>            <num>
1:                6.185269         10.39048
[1] 0.9314151


# Primer Corrida "Rapida y Liviana"
10 Iteraciones Bayesianas

- Experimento: 5940
- Semilla: 100007


PARAM$hypeparametertuning$hs <- makeParamSet
(
  makeNumericParam("learning_rate", lower = 0.05, upper = 0.2),

  makeIntegerParam("num_leaves", lower = 16L, upper = 128L),

  makeIntegerParam("max_depth", lower = 3L, upper = 8L),

  makeNumericParam("feature_fraction", lower = 0.6, upper = 1.0),

  makeIntegerParam("min_data_in_leaf", lower = 50L, upper = 1000L)
)

- Rango chico, para entrenar rápido.
- Ideal para datasets medianos (hasta 100k filas).
- max_depth limita el tamaño de los árboles para evitar overfitting.
- Con 10 iteraciones ya te da una buena base.

**Resultados:**

- learning_rate: 0.03173437
- num_leaves: 178
- feature_fraction: 0.5423336
- bagging_fraction: 0.7422481
- min_data_in_leaf: 548
- Kaggle: Muy malo. Subajusto y da 5 o 6 en el public Leaderboard

# Segunda Corrida

PARAM$hiperparametertuninghs <- makeParamSet(

  makeNumericParam("learning_rate", lower = 0.01, upper = 0.05),
  makeIntegerParam("num_leaves", lower = 8L, upper = 64L),
  makeNumericParam("feature_fraction", lower = 0.5, upper = 0.9),
  makeNumericParam("bagging_fraction", lower = 0.7, upper = 1.0),
  makeIntegerParam("min_data_in_leaf", lower = 20L, upper = 200L)
)

Resultados

    learning_rate: 0.01718392
    num_leaves: 35
    feature_fraction: 0.5969174
    bagging_fraction: 0.9411694
    min_data_in_leaf: 196
    max_depth: -1
    num_interactions: 1000

    Kaggle:
    - 100007: 55.267
    - 454547: 53.147
    - 737371: 50.527
    - 999971: 54.367
    - 999983: 52.247
    - Promedio en Public Leaderboard por Semilla: 53.111





# Tercer Corrida


PARAM$hypeparametertuning$hs <- makeParamSet(
  makeNumericParam("learning_rate", lower = 0.01, upper = 0.05),
  makeIntegerParam("num_leaves", lower = 16L, upper = 128L),
  makeNumericParam("feature_fraction", lower = 0.5, upper = 0.9),
  makeNumericParam("bagging_fraction", lower = 0.7, upper = 1.0),
  makeIntegerParam("min_data_in_leaf", lower = 20L, upper = 200L),
  makeNumericParam("lambda_l1", lower = 0, upper = 10),
  makeNumericParam("lambda_l2", lower = 0, upper = 10),
  makeNumericParam("min_gain_to_split", lower = 0, upper = 0.5),
  makeIntegerParam("max_depth", lower = -1L, upper = 10L)
)

Resultados


    learning_rate: 0.01390576
    num_leaves: 47
    feature_fraction: 0.5029553
    bagging_fraction: 0.8387449
    min_data_in_leaf: 158
    lambda_l1: 1.389795
    lambda_l2: 2.100884
    max_depth: 7 (uso -1)
    min_gain_to_split: 0.4506766
    num_interactions: 1000

    Probe con max_depth positivo y no me da buenos resultados (uso -1)

    Kaggle:
    - 100007: 54.827
    - 454547: 55.207
    - 737371: 55.227
    - 999971: 54.157
    - 999983: 55.677
    - Promedio en Public Leaderboard por Semilla: 55.019


# Cuarta Corrida


PARAM$hypeparametertuning$hs <- makeParamSet(
  makeNumericParam("learning_rate", lower = 0.01, upper = 0.03),
  makeIntegerParam("num_leaves", lower = 32L, upper = 96L),
  makeNumericParam("feature_fraction", lower = 0.6, upper = 0.9),
  makeNumericParam("bagging_fraction", lower = 0.7, upper = 1.0),
  makeIntegerParam("bagging_freq", lower = 0L, upper = 10L),
  makeIntegerParam("min_data_in_leaf", lower = 50L, upper = 250L),
  makeNumericParam("lambda_l1", lower = 0, upper = 10),
  makeNumericParam("lambda_l2", lower = 0, upper = 10),
  makeNumericParam("min_gain_to_split", lower = 0, upper = 0.3),
  makeIntegerParam("max_depth", lower = -1L, upper = 10L),
  makeNumericParam("min_sum_hessian_in_leaf", lower = 0.0001, upper = 10),
  makeNumericParam("scale_pos_weight", lower = 10, upper = 200)
)



Resultados

    learning_rate:    0.01005514
    num_leaves:       51
    feature_fraction: 0.7063844
    bagging_fraction: 0.9783509
    bagging_freq:     5
    min_data_in_leaf: 157
    lambda_l1:        9.632635
    lambda_l2:        7.882127
    max_depth:        0
    min_gain_to_split: 0.07555869
    min_sum_hessian_in_leaf: 6.185269
    scale_pos_weight: 10.39048
    num_interactions: 1000


Kaggle:

    - 100007:  
    - 454547:
    - 737371:
    - 999971:
    - 999983:
    - Promedio en Public Leaderboard por Semilla:


## 2.3  Produccion

### Final Training
Construyo el modelo final, que es uno solo, no hace ningun tipo de particion < training, validation, testing>]

In [ ]:
setwd("/content/buckets/b1/exp")
experimento <- paste0("exp", PARAM$experimento)
dir.create(experimento, showWarnings= FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento ))

#### Final Training Dataset

Aqui esta la gran decision de en qué meses hago el Final Training
<br> debo utilizar los mejores hiperparámetros que encontré en la optimización bayesiana

In [ ]:
# clase01
dataset[, clase01 := ifelse(clase_ternaria %in% c("BAJA+1", "BAJA+2"), 1L, 0L)]

In [ ]:
dataset_train <- dataset[foto_mes %in% c(202107)]

In [ ]:
# dejo los datos en el formato que necesita LightGBM

dtrain <- lgb.Dataset(
  data= data.matrix(dataset_train[, campos_buenos, with= FALSE]),
  label= dataset_train[, clase01]
)

#### Final Training Hyperparameters

In [ ]:
param_final <- modifyList(PARAM$lgbm$param_fijos,
  PARAM$out$lgbm$mejores_hiperparametros)

param_final

#### Training
Genero el modelo final, siempre sobre TODOS los datos de  final_train, sin hacer ningun tipo de undersampling de la clase mayoritaria

In [ ]:
# este punto es muy SUTIL  y será revisado en la Clase 05

param_normalizado <- copy(param_final)
param_normalizado$min_data_in_leaf <-  round(param_final$min_data_in_leaf / PARAM$trainingstrategy$undersampling)

In [ ]:
  # entreno LightGBM

  modelo_final <- lgb.train(
    data= dtrain,
    param= param_normalizado
  )

In [ ]:
# ahora imprimo la importancia de variables

tb_importancia <- as.data.table(lgb.importance(modelo_final))
archivo_importancia <- "impo.txt"

fwrite(tb_importancia,
  file= archivo_importancia,
  sep= "\t"
)

In [ ]:
# grabo a disco el modelo en un formato para seres humanos ... ponele ...

lgb.save(modelo_final, "modelo.txt" )

### Scoring

Aplico el modelo final a los datos del futuro

In [ ]:
# aplico el modelo a los datos sin clase
dfuture <- dataset[foto_mes == 202109]

# aplico el modelo a los datos nuevos
prediccion <- predict(
  modelo_final,
  data.matrix(dfuture[, campos_buenos, with= FALSE])
)

#### Tabla Prediccion

In [ ]:
# tabla de prediccion

tb_prediccion <- dfuture[, list(numero_de_cliente)]
tb_prediccion[, prob := prediccion ]

# grabo las probabilidad del modelo
fwrite(tb_prediccion,
  file= "prediccion.txt",
  sep= "\t"
)

Kaggle Competition Submit

In [ ]:
# genero archivos con los  "envios" mejores
# suba TODOS los archivos a Kaggle

# ordeno por probabilidad descendente
setorder(tb_prediccion, -prob)

dir.create("kaggle")

for (envios in PARAM$kaggle$cortes) {

  tb_prediccion[, Predicted := 0L] # seteo inicial a 0
  tb_prediccion[1:envios, Predicted := 1L] # marclo los primeros

  archivo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_", envios, ".csv")

  # grabo el archivo
  fwrite(tb_prediccion[, list(numero_de_cliente, Predicted)],
    file= archivo_kaggle,
    sep= ","
  )

  # subida a Kaggle, armo la linea de comando
  comando <- "kaggle competitions submit"
  competencia <- paste("-c", PARAM$kaggle$competencia)
  arch <- paste( "-f", archivo_kaggle)

  mensaje <- paste0("-m 'envios=", envios,
  "  semilla=", PARAM$semilla_primigenia,
    "'" )

  linea <- paste( comando, competencia, arch, mensaje)

  salida <- system(linea, intern=TRUE) # el submit a Kaggle
  cat(salida, "\n")
  Sys.sleep(45)
}

In [ ]:
write_yaml( PARAM, file="PARAM.yml")

In [ ]:
format(Sys.time(), "%a %b %d %X %Y")

Finalmente usted deberá cargar el resultado de su corrida en la Google Sheet Colaborativa,  hoja **TareaHogar-05**
<br> Siéntase libre de agregar las columnas que hagan falta a la planilla

Seguramente usted realice varias corridas de este script con distintos conjuntos de hiperparámetros, siempre cambiandole el nombre al script  y también cambiando el nombre del experimento,  deberá TODAS esas corridas en distintas lineas de la  Google Sheet Colaborativa, hoja **TareaHogar-05**

Siéntase libre de agregar columnas a la hoja **TareaHogar-05**  en caso de ser necesario.